In [1]:
import json
import multiprocessing
import os
import re
import tempfile
from collections import Counter
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import plyvel
import polars as pl
import whoosh
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
from whoosh.analysis import StandardAnalyzer

In [2]:
df = pl.read_parquet("/kaggle/input/uspto-boolean-search-optimization/patent_metadata.parquet")
df.head(1)

publication_number,publication_date,filing_date,family_id,cpc_codes
str,datetime[μs],datetime[μs],f64,list[str]
"""US-1-A""",1836-07-13 00:00:00,null,2.060279e6,"[""B61C11/04""]"


In [3]:
from typing import Dict, List

cpc2patents: Dict[str, List[str]] = {}
for cpc, patent in tqdm(zip(df["cpc_codes"], df["publication_number"]), total=len(df)):
    for code in cpc:
        if code not in cpc2patents:
            cpc2patents[code] = []
        cpc2patents[code].append(patent)

100%|██████████| 13307751/13307751 [25:58<00:00, 8537.28it/s]  


In [4]:
# データベースの作成
db = plyvel.DB("db", create_if_missing=True)
for cpc, patents in tqdm(cpc2patents.items()):
    db.put(cpc.encode(), json.dumps(patents).encode())

100%|██████████| 265889/265889 [02:45<00:00, 1606.95it/s] 


In [5]:
# 速度を確認
for cpc in tqdm(cpc2patents):
    patents = json.loads(db.get(cpc.encode()).decode())
patents

100%|██████████| 265889/265889 [00:10<00:00, 25393.23it/s]


['US-RE48165-E']

In [6]:
db.close()